In [1]:
from bs4 import BeautifulSoup 
import requests


url = 'https://www.craigslist.org/about/sites'
headers = {'referer': 'https://usa.fishermap.org/', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
r = requests.get(url,headers=headers)

soup = BeautifulSoup(r.content)

In [2]:
non_cont_us_str = str(soup).split('\n<h4>Territories<')[1]

urls = list()

for t in soup.findAll('li'):
    if t.find('a'):
        s = t.find('a').attrs['href']
        if s.endswith('.org') and s not in non_cont_us_str:
            urls.append(s)

In [3]:
def get_state_urls(state):
    url = f'https://geo.craigslist.org/iso/us/{state}'
    headers = {'referer': 'https://usa.fishermap.org/', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
    r = requests.get(url,headers=headers)

    soup = BeautifulSoup(r.content)

    badwords = [
        'help',
        'safety',
        'privacy',
        'feedback',
        'terms',
        'about',
        'craigslist app',
        'cl is hiring'
     ]

    urls = list()

    for i in [x.find('a') for x in soup.findAll('li')]:
        if i and i.string not in badwords:
            urls.append(i.attrs['href'])
            
    
    return urls

In [6]:
from uszipcode import SearchEngine
search = SearchEngine()
result = search.by_population(lower=0, upper=1000000000, returns = 1000000000)

C:\Users\15714\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
home = search.by_zipcode('24016')

In [8]:
import math

data = list()
for zipcode in result:
    data.append(
        {'major_city': zipcode.major_city,
         'state': zipcode.state,
         'lat': zipcode.lat,
         'lon': zipcode.lng,
         'dist': math.sqrt((zipcode.lat - home.lat)**2 + (zipcode.lng - home.lng)**2)
        }
    )

In [9]:
import pandas as pd
df = pd.DataFrame(data = data)

df = df.sort_values('dist')

,major_city,state,lat,lon,dist
0,Chicago,IL,41.78,-87.71,8.975394
1,El Paso,TX,31.78,-106.30,26.915843
2,Corona,NY,40.74,-73.85,7.017899
3,Norwalk,CA,33.90,-118.07,38.268673
4,Los Angeles,CA,34.00,-118.25,38.439341
...,...,...,...,...,...
28638,Glentana,MT,48.84,-106.24,28.723318
28639,Mohler,WA,47.36,-118.35,39.703502
28640,Fairbanks,AK,65.39,-148.33,73.936181
28641,Waldron,KS,37.00,-98.18,18.231999


In [23]:
# 8 hour drive max entry
zips = df[df['dist'] < 8]
set(zips['state'])

{'AL',
 'CT',
 'DC',
 'DE',
 'FL',
 'GA',
 'IL',
 'IN',
 'KY',
 'MD',
 'MI',
 'NC',
 'NJ',
 'NY',
 'OH',
 'PA',
 'SC',
 'TN',
 'VA',
 'WV'}